In [22]:
# Impor modul os dan shutil
import os
import shutil

# Tentukan nama file dan ekstensi lama
filename = 'ecoli.data'
old_ext = '.data'

# Tentukan ekstensi baru
new_ext = '.csv'

# Pisahkan nama file dan ekstensi lama
name, ext = os.path.splitext(filename)

# Periksa apakah ekstensi lama sesuai dengan yang diharapkan
if ext == old_ext:
    # Buat nama file baru dengan ekstensi baru
    new_filename = name + new_ext
    # Salin file lama ke file baru
    shutil.copy(filename, new_filename)
    # Hapus file lama
    os.remove(filename)
    # Cetak pesan konfirmasi
    print('File {} berhasil diubah menjadi {}'.format(filename, new_filename))
else:
    # Cetak pesan kesalahan
    print('File {} tidak memiliki ekstensi {}'.format(filename, old_ext))


File ecoli.data berhasil diubah menjadi ecoli.csv


In [23]:
# Impor modul csv
import csv

# Buka file csv dengan mode 'r' (read)
with open('ecoli.csv', 'r') as f:
    # Buat objek reader untuk membaca file
    reader = csv.reader(f)
    # Buat daftar kosong untuk menyimpan data yang sudah dipisah
    data = []
    # Iterasi setiap baris di file
    for row in reader:
        # Pisahkan baris dengan spasi sebagai pemisah
        row = row[0].split(' ')
        # Tambahkan baris yang sudah dipisah ke daftar data
        data.append(row)

# Buka file baru dengan mode 'w' (write)
with open('ecoli_new.csv', 'w') as f:
    # Buat objek writer untuk menulis file
    writer = csv.writer(f)
    # Tulis semua baris dari daftar data ke file baru
    writer.writerows(data)

# Cetak pesan konfirmasi
print('File ecoli.csv berhasil dimodifikasi dan disimpan sebagai ecoli_new.csv')


File ecoli.csv berhasil dimodifikasi dan disimpan sebagai ecoli_new.csv


In [24]:
# Impor modul math dan csv
import math

# Baca file data Ecoli.csv
data = []
with open('ecoli.csv', 'r') as f:
    for line in f:
        row = line.strip().split(',')
        data.append(row)

# Bagi data menjadi dua set: pelatihan dan pengujian
train_ratio = 0.8 # rasio 80:20
train_size = int(len(data) * train_ratio)
train_data = data[:train_size]
test_data = data[train_size:]

# Hitung probabilitas prior untuk setiap kelas protein
prior = {} # kamus untuk menyimpan probabilitas prior
classes = ['cp', 'im', 'pp', 'imU', 'om', 'omL', 'imL', 'imS'] # daftar kelas protein
for c in classes:
    count = 0 # hitung jumlah sampel dengan kelas c
    for row in train_data:
        if row[-1] == c: # jika label sama dengan c
            count += 1
    prior[c] = count / len(train_data) # hitung probabilitas prior sebagai frekuensi relatif

# Hitung probabilitas kondisional untuk setiap fitur dan kelas protein
conditional = {} # kamus untuk menyimpan probabilitas kondisional
features = ['mcg', 'gvh', 'lip', 'chg', 'aac', 'alm1', 'alm2'] # daftar fitur
for c in classes:
    conditional[c] = {} # kamus bersarang untuk menyimpan probabilitas kondisional untuk kelas c
    for f in features:
        conditional[c][f] = {} # kamus bersarang lagi untuk menyimpan probabilitas kondisional untuk fitur f dan kelas c
        values = [] # daftar untuk menyimpan nilai fitur f untuk semua sampel dengan kelas c
        for row in train_data:
            if row[-1] == c: # jika label sama dengan c
                values.append(float(row[features.index(f)])) # tambahkan nilai fitur f ke daftar
        mean = sum(values) / len(values) # hitung rata-rata nilai fitur f
        variance = sum([(x - mean)**2 for x in values]) / len(values) # hitung varians nilai fitur f
        std = math.sqrt(variance) # hitung standar deviasi nilai fitur f
        conditional[c][f]['mean'] = mean # simpan rata-rata ke dalam kamus
        conditional[c][f]['std'] = std # simpan standar deviasi ke dalam kamus

# Fungsi untuk menghitung probabilitas normal
def normal_prob(x, mean, std):
    return (1 / (math.sqrt(2 * math.pi) * std)) * math.exp(-((x - mean)**2 / (2 * std**2)))

# Fungsi untuk menghitung probabilitas posterior
def posterior_prob(row):
    posterior = {} # kamus untuk menyimpan probabilitas posterior
    for c in classes:
        posterior[c] = math.log(prior[c]) # inisialisasi dengan probabilitas prior
        for f in features:
            x = float(row[features.index(f)]) # ambil nilai fitur f dari baris data
            mean = conditional[c][f]['mean'] # ambil rata-rata fitur f dan kelas c dari kamus
            std = conditional[c][f]['std'] # ambil standar deviasi fitur f dan kelas c dari kamus
            posterior[c] += math.log(normal_prob(x, mean, std)) # tambahkan logaritma probabilitas normal ke probabilitas posterior
    return posterior

# Prediksi kelas protein untuk setiap sampel di set pengujian
predictions = [] # daftar untuk menyimpan prediksi
for row in test_data:
    posterior = posterior_prob(row) # hitung probabilitas posterior untuk baris data
    max_prob = max(posterior.values()) # cari nilai maksimum dari probabilitas posterior
    for c in classes:
        if posterior[c] == max_prob: # jika probabilitas posterior sama dengan nilai maksimum
            predictions.append(c) # tambahkan kelas c ke daftar prediksi
            break

# Hitung akurasi prediksi
correct = 0 # hitung jumlah prediksi yang benar
for i in range(len(test_data)):
    if predictions[i] == test_data[i][-1]: # jika prediksi sama dengan label sebenarnya
        correct += 1
accuracy = correct / len(test_data) # hitung akurasi sebagai rasio prediksi yang benar terhadap jumlah total prediksi
print('Akurasi: {:.2f}%'.format(accuracy * 100)) # cetak akurasi dalam persen dengan dua angka desimal


ZeroDivisionError: division by zero

In [25]:
import csv
import math
import random

# Fungsi untuk memisahkan data menjadi training dan testing
def split_dataset(dataset, split_ratio):
    train_size = int(len(dataset) * split_ratio)
    train_set = []
    test_set = list(dataset)
    while len(train_set) < train_size:
        index = random.randrange(len(test_set))
        train_set.append(test_set.pop(index))
    return train_set, test_set

# Fungsi untuk menghitung mean dari list angka
def mean(numbers):
    return sum(numbers) / float(len(numbers))

# Fungsi untuk menghitung standard deviation dari list angka
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)

# Fungsi untuk memproses dataset dan menghitung statistik untuk setiap fitur
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column)) for column in zip(*[row[:-1] for row in dataset])]
    return summaries

# Fungsi untuk memisahkan dataset berdasarkan kelas
def separate_by_class(dataset):
    separated = {}
    for i in range(1, len(dataset)):
        x = dataset[i]
        if x[-1] not in separated:
            separated[x[-1]] = []
        separated[x[-1]].append(x)
    return separated

# Fungsi untuk memproses dataset berdasarkan kelas dan menghitung statistik untuk setiap kelas
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = {}
    for class_value, instances in separated.items():
        summaries[class_value] = summarize_dataset(instances)
    return summaries

# Fungsi untuk menghitung Gaussian Probability Density Function
def calculate_probability(x, mean, stdev):
    exponent = math.exp(-((x - mean) ** 2) / (2 * stdev ** 2))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent

# Fungsi untuk memprediksi kelas dari data uji
def predict(summaries, input_vector):
    probabilities = {}
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = 1
        for i in range(len(class_summaries)):
            mean, stdev = class_summaries[i]
            x = input_vector[i]
            probabilities[class_value] *= calculate_probability(x, mean, stdev)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

# Fungsi untuk mengukur akurasi dari model
def get_accuracy(test_set, predictions):
    correct = 0
    for i in range(len(test_set)):
        if test_set[i][-1] == predictions[i]:
            correct += 1
    return (correct / float(len(test_set))) * 100.0

# Load dataset Iris
def load_dataset(filename):
    dataset = []
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Lewati baris header
        for row in csv_reader:
            dataset.append([float(x) for x in row[:-1]] + [row[-1]])
    return dataset

# Main program
filename = 'ecoli.csv'
dataset = load_dataset(filename)
split_ratio = 0.7
training_set, testing_set = split_dataset(dataset, split_ratio)

summaries = summarize_by_class(training_set)

predictions = []
for test_instance in testing_set:
    prediction = predict(summaries, test_instance[:-1])
    predictions.append(prediction)

accuracy = get_accuracy(testing_set, predictions)
print(f'Accuracy: {accuracy:.2f}%')

ZeroDivisionError: float division by zero

: 